In [19]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd 
import networkx as nx 
import nupack as nu
from random import choice as choicerandom
from numpy.random import choice as choicenumpy
from hdna import * 

In [21]:
M = Model('dna', '3D', min_nucleation=2, sliding_cutoff=3, stacking='nostacking')
# A = Strand(M, 'GGGGGGGGGGGGGGGGG')
A = Strand(M, 'ACCAAACCACCAAC')
# A = Strand(M, 'ATGATGTTCCGCTTT')
B = A.complementary()

In [68]:
G = Geometry(120,270)
C = Chamber(M, A, B)
N = Kinetwork(M, A, B)
K = Kinetics(M, N, G)
S = Simulator(M, N, K)

In [52]:
for slide in C.offcores:
    for e in slide.backfray: 
        print(e.structure, e.G)
    print('\n')



(((((.........+.)))))........ -6.925764238903476
.(((..........+..)))......... -4.120449068824481
..(...........+...).......... -0.4881872600222883




((((..........+.....))))..... -5.681043942272813
.((...........+......))...... -2.546662191507054




(....((((.(...+).))))....)... -0.38376681830054116
.....((((.....+..))))........ -5.681043942272813
......((......+...))......... -2.546662191507054






...(....((((.(+...).))))....) -2.450460353686646
........((((..+.....))))..... -5.681043942272813
.........((...+......))...... -2.546662191507054




.....((((.....+..........)))) -5.681043942272813
......((......+...........)). -2.546662191507054




........(((((.+.........))))) -6.925764238903476
.........(((..+..........))). -4.120449068824481
..........(...+...........).. -0.4881872600222883






In [78]:
import numpy as np 
slide = C.offcores[5]
maxG = np.argmin(slide.gbackfray)
print(maxG, slide.gbackfray[maxG])
print(*zip(slide.backfray, slide.sbackfray, slide.gbackfray), sep='\n')



1 -5.681043942272813
(<hdna.complex.Sliding object at 0x7fdcd0115090>, '(....((((.(...+).))))....)...', -0.38376681830054116)
(<hdna.complex.Zippo object at 0x7fdcd0117cd0>, '.....((((.....+..))))........', -5.681043942272813)
(<hdna.complex.Zippo object at 0x7fdcc1bdc460>, '......((......+...)).........', -2.546662191507054)


In [48]:
DG = S.DiGraph()
DF = pd.DataFrame(DG.nodes.data('object'))
DF = DF.set_axis(['structure', 'object'], axis=1)
DF['gibbs'] = [obj.G for obj in DF['object']]
PF = DF[DF['gibbs'] < 0]
for s, data in PF.iterrows():
    if type(data['object']) == Sliding:
        print(s)
        print(data['object'].structure)

14
((............+))............
15
............((+............))
16
(((((.........+.)))))........
17
........(((((.+.........)))))
18
..(.(((.......+))).).........
19
.........(.(((+.......))).)..
20
((((..........+.....)))).....
21
.....((((.....+..........))))
24
(....((((.(...+).))))....)...
25
...(....((((.(+...).))))....)


In [49]:
def get_ix(string, char):
            indices = []
            for i, e in enumerate(list(string)):
                if e == char:
                    indices.append(i)
            return indices 

In [26]:
import itertools as it 
a = '..............)))))))............'
ix = get_ix(a, ')')
for d, i in it.zip_longest(ix[:int(len(ix)/2)], ix[int(len(ix)/2):][::-1]):
    try: a = a[:d] + '.' + a[d+1:]
    except TypeError: pass 
    try: a = a[:i] + '.' + a[i+1:]
    except TypeError: pass
    print(a)

...............))))).............
................)))..............
.................)...............
.................................


In [27]:
slide = PF.loc[25,'object']
slide.backfraying_trajectory()
for e in slide.backfrayings:
    print(e)
print(slide.structure)

...(....((((.(+...).))))....)


In [28]:
nupackmodel = nu.Model(
    material='dna', 
    ensemble='nostacking', 
    celsius=25, 
    sodium=1, 
    magnesium=0) 

nuStrand1 = nu.Strand(A.sequence, name = 'a')
nuStrand2 = nu.Strand(B.sequence, name = 'b') # An inversion here is needed because in this program strands are defined as 5-3 against 3-5 but in NUPACK all strands are defined 5-3 and the program takes care to turn them around and so on
nuStructure = nu.Structure('.......((........+........)).......')
dG = nu.structure_energy(strands=[nuStrand1,nuStrand2], structure=nuStructure, model=nupackmodel)
# dGloop = nupackmodel.loop_energy(loop='AAA+TTT', structure='(((+)))')
dG

RuntimeError: C++: : "pair list length doesn't match sequence length" (State.cc, line 27)
len(pairs) = 34
n = 32
